In [1]:
import json
import urllib.request
import pandas as pd

#Request data from the API
BASE_URL = 'https://data.chhs.ca.gov'
#Override default limit of 100 by explicitly setting to its length (905)
ENDPOINT = '/api/3/action/datastore_search?resource_id=8dd44ad7-9e78-4c1a-9910-f58a47e402af&limit=905'  
fileobj = urllib.request.urlopen(BASE_URL+ENDPOINT)
response_dict = json.loads(fileobj.read())

#Resulting dataset is the CalHHS 30-Day Hospital Readmission Rate
#rr_df = pd_DataFrame(response_dict)
#rr_df.head(5)
rr_df = pd.DataFrame(response_dict['result']['records']).set_index('_id') 
rr_df.head(5)

,Year,Strata,Strata Name,County,Total Admits (ICD-9),30-day Readmits (ICD-9),30-day Readmission Rate (ICD-9),Total Admits (ICD-10),30-day Readmits (ICD-10),30-day Readmission Rate (ICD-10)
_id,,,,,,,,,,
1,2011,Overall,Overall,State,1948641,272268,14%,None,None,None
2,2011,Age,18 to 44 years,State,326070,36855,11.30%,None,None,None
3,2011,Age,45 to 64 years,State,659801,90891,13.80%,None,None,None
4,2011,Age,65 years and above,State,962771,144522,15%,None,None,None
5,2011,Sex,Male,State,901776,132417,14.70%,None,None,None


In [2]:
#All of the columns are objects, in order to perform required cleaning
#it's necessary to isolate the categorical columns
cat_cols = ['Strata', 'Strata Name', 'County']
#and change the dtype of the numeric columns
num_cols = rr_df.columns.difference(cat_cols)
# Strip non-numeric characters like % and convert to float
rr_df[num_cols] = (
    rr_df[num_cols]
    #Remove all non-numeric characters
    .replace(r"[^\d.]", "", regex=True)
    #Convert to numeric, replace invalid entries with NaN
    .apply(pd.to_numeric, errors="coerce")  
)

International Classification of Diseases (ICD) is a standard tool for reporting medical diagnoses and procedures.

ICD-10 replaced ICD-9 October 1, 2015. With this in mind the columns corresponding to each ICD may be consolidated into a single column.

The dates preceeding this deadline are ICD-9 and the dates which follow are ICD-10.

In [3]:
# Consolidate columns and create a new "ICD Version" column
def consolidate_icd(row):
    # Determine ICD version based on the year
    if row['Year'] <= 2015:
        # ICD-9 applies
        return {
            'ICD Version': 'ICD-9',
            'Total Admits': row['Total Admits (ICD-9)'],
            '30-day Readmits': row['30-day Readmits (ICD-9)'],
            '30-day Readmission Rate': row['30-day Readmission Rate (ICD-9)']
        }
    else:
        # ICD-10 applies
        return {
            'ICD Version': 'ICD-10',
            'Total Admits': row['Total Admits (ICD-10)'],
            '30-day Readmits': row['30-day Readmits (ICD-10)'],
            '30-day Readmission Rate': row['30-day Readmission Rate (ICD-10)']
        }

# Apply the function to each row
consolidated = rr_df.apply(lambda row: pd.Series(consolidate_icd(row)), axis=1)

# Add the new consolidated data to the original DataFrame
rr_df['ICD Version'] = consolidated['ICD Version']
rr_df['Total Admits (Consolidated)'] = consolidated['Total Admits']
rr_df['30-day Readmits (Consolidated)'] = consolidated['30-day Readmits']
rr_df['30-day Readmission Rate (Consolidated)'] = consolidated['30-day Readmission Rate']

# Drop the original ICD-9 and ICD-10 columns
columns_to_drop = [
    'Total Admits (ICD-9)', '30-day Readmits (ICD-9)', '30-day Readmission Rate (ICD-9)',
    'Total Admits (ICD-10)', '30-day Readmits (ICD-10)', '30-day Readmission Rate (ICD-10)'
]
rr_df.drop(columns=columns_to_drop, inplace=True)

# Display the updated DataFrame
rr_df.head()

      

,Year,Strata,Strata Name,County,ICD Version,Total Admits (Consolidated),30-day Readmits (Consolidated),30-day Readmission Rate (Consolidated)
_id,,,,,,,,
1,2011,Overall,Overall,State,ICD-9,1948641.0,272268.0,14.0
2,2011,Age,18 to 44 years,State,ICD-9,326070.0,36855.0,11.3
3,2011,Age,45 to 64 years,State,ICD-9,659801.0,90891.0,13.8
4,2011,Age,65 years and above,State,ICD-9,962771.0,144522.0,15.0
5,2011,Sex,Male,State,ICD-9,901776.0,132417.0,14.7


In [4]:
#Unpack Strata into individual columns whose rows are Strata Name

# Create a dictionary to hold the new columns
strata_columns = {}

# Iterate through unique 'Strata' values to add them as new columns
for strata in rr_df['Strata'].unique():
    strata_columns[strata] = rr_df.apply(
        lambda row: row['Strata Name'] if row['Strata'] == strata else None, axis=1
    )

# Add the new columns to the DataFrame
for strata, values in strata_columns.items():
    rr_df[strata] = values

# Fill any NaN values with 0 
rr_df.fillna(0, inplace=True)

rr_df.head(5)

,Year,Strata,Strata Name,County,ICD Version,Total Admits (Consolidated),30-day Readmits (Consolidated),30-day Readmission Rate (Consolidated),Overall,Age,Sex,Race-Ethnicity,Expected Payer
_id,,,,,,,,,,,,,
1,2011,Overall,Overall,State,ICD-9,1948641.0,272268.0,14.0,Overall,0,0,0,0
2,2011,Age,18 to 44 years,State,ICD-9,326070.0,36855.0,11.3,0,18 to 44 years,0,0,0
3,2011,Age,45 to 64 years,State,ICD-9,659801.0,90891.0,13.8,0,45 to 64 years,0,0,0
4,2011,Age,65 years and above,State,ICD-9,962771.0,144522.0,15.0,0,65 years and above,0,0,0
5,2011,Sex,Male,State,ICD-9,901776.0,132417.0,14.7,0,0,Male,0,0


In [5]:
rr_df.to_csv('CalHHS_30-Day_Readmission_Rate.csv')


In [ ]:
# Step 1: Navigate to the repository directory
%cd /PyCharmProjects/Predicting_Patient_Readmission

# Step 2: Copy the notebook to the repository (if needed)
!cp /path/to/CalHHS_30-Day_Hospital_Readmission_Rate.ipynb /path/to/Predicting_Patient_Readmission/

# Step 3: Initialize the Git repository (if not already done)
!git init

# Step 4: Add the remote origin (if not already set)
!git remote add origin https://github.com/jihbr/Predicting_Patient_Readmission.git

# Step 5: Stage the notebook file
!git add CalHHS_30-Day_Hospital_Readmission_Rate.ipynb

# Step 6: Commit the file
!git commit -m "Add Jupyter Notebook for CalHHS 30-Day Hospital Readmission Rate analysis"

# Step 7: Push the file to GitHub
!git push origin main
